In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input, GRU

In [ ]:
stock = yf.Ticker('TATAMOTORS.NS')
data = stock.history(period = 'max')
data

In [ ]:
close = data['Close'].values
close=close.reshape(-1,1)
close.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))
scaled_data.shape

In [ ]:
train_data = scaled_data[:].reshape(-1,1)
train_data.shape

In [ ]:
prediction_days = 100

In [ ]:
x_train = []
y_train = []

In [ ]:
for x in range(prediction_days, len(train_data)):
  x_train.append(train_data[x-prediction_days:x, 0])
  y_train.append(train_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()

model.add(Input(shape = (x_train.shape[1],1)))

model.add(LSTM(units = 100, return_sequences=True))
model.add(Dropout(0.2))

model.add(GRU(units=100, return_sequences=True))
model.add(Dropout(0.2))

model.add(GRU(units=100, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs = 100, batch_size = 64)

In [ ]:
pred = model.predict(x_train)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_train, color = 'red', label = 'Actual')
plt.plot(pred, color = 'green', label = 'LSTM-GRU Prediction')
plt.ylabel('TATAMOTORS Price in Rs')
plt.xlabel('Time in Integer')
plt.title('TATAMOTORS Share Prices')
plt.legend()
plt.show()

In [ ]:
stock_test = yf.Ticker('TATASTEEL.NS')
data_test = stock_test.history(period = 'max')
data_test

In [ ]:
scaled_data = scaler.fit_transform(data_test['Close'].values.reshape(-1,1))
test_data = scaled_data[:].reshape(-1,1)
test_data.shape

In [ ]:
x_test = []
y_test = []
prediction_days = 100

In [ ]:
for x in range(prediction_days, len(test_data)):
  x_test.append(test_data[x-prediction_days:x, 0])
  y_test.append(test_data[x, 0])

x_test, y_test = np.array(x_test), np.array(y_test)

x_test.shape, y_test.shape

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test.shape

In [ ]:
prediction_price = model.predict(x_test)

In [ ]:
y_test = y_test.reshape(-1,1)
y_test.shape

In [ ]:
y_test = scaler.inverse_transform(y_test)
prediction_price = scaler.inverse_transform(prediction_price)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_test, color = 'red', label = 'Actual')
plt.plot(prediction_price, color = '#89DA59', label = 'LSTM-GRU Prediction')
plt.ylabel('TATASTEEL Price in Rs')
plt.xlabel('Time in Integer')
plt.title('TATASTEEL Share Prices')
plt.legend()
plt.show()